In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('BBGM_League_2_all_seasons_Average_Stats.csv')

In [ ]:
y = df.iloc[:,-15:]
X = df.iloc[:,11:-17]
y = y[(X['AST%'] >0) & (X['AST%'] < 100)]
X = X[(X['AST%'] >0) & (X['AST%'] < 100)]
X['MP'] = df.MP

df.columns,X.shape

In [ ]:
for name in ['FG','FGA','3P',"3PA",'FT','FTA',\
             'ORB','DRB','TRB','AST','TOV','STL',"Blk",\
             'PF',"PTS",'OWS','DWS']:
    X[name + 'p36'] = 36* df[name]/np.maximum(1,df.MP)
    X[name + 'p100'] = X[name + 'p36']*4/3

X['3PtP'] = (2/(1+np.exp(-X['3PAp100']))-1)*X['3P%']/100
X['Creation'] = X['ASTp100']*0.1843+(X['PTSp100']+X['TOVp100'])*0.0969-2.3021*X['3PtP']+0.0582*(X['ASTp100']*(X['PTSp100']+X['TOVp100'])*X['3PtP'] )-1.1942
X['Load'] = (X['ASTp100']-(0.38*X['Creation'])*0.75)+X['FGAp100']+X['FTAp100']*0.44+X['Creation']+X['TOVp100']
X['cTOV'] = X['TOVp100']/X['Load']
#X['DPM'] = X['Blkp100']*0.802+X['DRBp100']*0.42-4.7-0.07551*X['PFp100']+1.597019*X['STLp100']-0.26385*X['TOVp100']
#X['OPM'] = -8.57647+0.6111*X['PTSp100']-0.33918*(0.44*X['FTAp100']+X['FGAp100'])+0.440814*X['FTAp100']+0.379745*X['3PAp100']+0.634044*X['ASTp100']+0.77827*X['ORBp100']-1.08855*X['TOVp100']+0.26262*X['STLp100']

#X['PassP'] = ((X['ASTp100']-(0.38*X['Creation']))*0.752+ X['Creation'] + X['TOVp100']) ** 0.67

X = X[[_ for _ in X.columns if '%' in _ or _[-1]=='r' or 'FGP' in _ or "+/-" in _ or 'p36' in _ or _ in ['OPM','DPM','cTOV','3PtP','Load','Creation','PER','MP']]]
X.columns

In [ ]:
#_ =  plt.hist(X['OPM'],150)

In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing


In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=2,interaction_only=False)
scalerX = preprocessing.RobustScaler()
scalery = preprocessing.StandardScaler()
prescale_X = scalerX.fit_transform(X)
prescale_y = scalery.fit_transform(y)
prescale_X = fexp.fit_transform(prescale_X)

In [ ]:
trials = 1
ts = []
for i in range(trials):
    #clf = neural_network.MLPRegressor((36,5,24,36),'tanh',solver='adam',max_iter=1000)
    clf = neural_network.MLPRegressor((),'identity',solver='lbfgs')
    clf.fit(prescale_X,prescale_y)
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    err = np.linalg.norm(yt-y)
    ts.append((err,clf))

In [ ]:
ts = sorted(ts)
print(ts[0][0])
clf = ts[0][1]

In [ ]:
col_names = X.columns
col_names = fexp.get_feature_names(X.columns)

for i,c in enumerate(y.columns):
    coeffs = clf.coefs_[-1][:,i]
    v = np.argsort(coeffs)[::-1]
    print(c)
    for i2 in v:
        if abs(coeffs[i2]) < 0.1:
            continue
        print('\t{:25s}\t{:.2f}'.format(col_names[i2],coeffs[i2]))


In [ ]:
tyear = 2019
tables =np.load('tables_{}.pkl'.format(tyear))
teams = tables.keys()

In [ ]:
player_stats = {}
table_columns = {}
for team in tables:
    team_tables =  tables[team]
    for table_name in team_tables:
        if table_name in ['draft-rights','team_and_opponent']:
            continue
        table = team_tables[table_name].fillna(0)
        #print(table_name)
        #print(table.index)
        for row in table.itertuples():
            name = row[0]
            if name == 'Team Totals':
                continue
            player_table = player_stats.get(name,{})
            player_row = player_table.get(table_name,[])
            player_row = player_row + [row]
            
            player_table[table_name] = player_row
            player_stats[name] = player_table
        table_columns[table_name] = table.columns


In [ ]:
table_mask = {}
for table in table_columns:
    table_mask[table] = [_.strip() !=''  for _ in table_columns[table] ]
    table_columns[table] = [_ for _ in table_columns[table] if _.strip() != '']
for player in player_stats:
    for table_in in player_stats[player]:
        if 'on_off' in table_in or 'salaries' in table_in:
            continue
        if len(player_stats[player][table_in]) > 1:
            pass
            #print(player,table_in,'MP' in player_stats[player][table_in][0]._fields)
            #print(player_stats[player][table_in][0])

In [ ]:
for player in player_stats:
    for tt in player_stats[player]:
        new_rows = []
        for tablet in player_stats[player][tt]:
            vector = [_ if _ != '' else '0.5' for _ in tablet[1:]]
            vector = [(float(_.replace('%',''))/100 if type(_) == str and'%' in _ else _) for _ in vector]
            if 'on_off' in tt:
                vector = vector[1:]
            if 'contracts' in tt:
                vector = vector[1:-2]
            if tt in ['salaries2','contracts']:
                vector = [_.replace(',','').replace('$','') for _ in vector]
            try:
                v2 = np.array(vector).astype(np.float)
            except:
                v2 = vector
            new_rows.append(vector)
        a = np.array(new_rows)
        
        if 'MP' in table_columns[tt] and not tt in ['pbp','on_off','on_off_p']:
            a = a.astype(np.float)
            try:
                mins = a[:,table_columns[tt].index('MP')].reshape((-1,1))
                new_rows = ((a.T @ mins)/mins.sum()).T
                a = new_rows
            except:
                print(tt,a.shape,player,a,mins)
                raise

        player_stats[player][tt] = a


In [ ]:
player_stats['Ben Simmons'].keys()

In [ ]:
player_vectors = []
player_names = []
for name in player_stats:
    #name = 'Jimmy Butler'
    #name = 'Ben Simmons'
    if not 'totals' in player_stats[name]:
        continue
    player_names.append(name)
    #for stat in X.columns:
    d=    {'FG%':player_stats[name]['totals'][0][6]*100,
        '3P%':player_stats[name]['totals'][0][9]*100,
        'MP':player_stats[name]['per_game'][0][3],
        'FT%':player_stats[name]['totals'][0][16]*100,
        'AtRimFGP':player_stats[name]['shooting'][0][11]*100,
        'LowPostFGP':player_stats[name]['shooting'][0][11]*100, #12
        'MidRangeFGP':player_stats[name]['shooting'][0][13]*100,
        'TS%':player_stats[name]['advanced'][0][4]*100,
        '3PAr':player_stats[name]['advanced'][0][5]*100,
        'FTr':player_stats[name]['advanced'][0][6]*100,
        'ORB%':player_stats[name]['advanced'][0][7],
        'DRB%':player_stats[name]['advanced'][0][8],
        'TRB%':player_stats[name]['advanced'][0][9],
        'AST%':player_stats[name]['advanced'][0][10],
        'STL%':player_stats[name]['advanced'][0][11],
        'BLK%':player_stats[name]['advanced'][0][12],
        'TOV%':player_stats[name]['advanced'][0][13],
        'USG%':player_stats[name]['advanced'][0][14],
        '+/-':player_stats[name]['advanced'][0][22],
        'FGp36':player_stats[name]['per_minute'][0][4],
        'FGAp36':player_stats[name]['per_minute'][0][5],
        '3Pp36':player_stats[name]['per_minute'][0][6],
        '3PAp36':player_stats[name]['per_minute'][0][7],
        'FTp36':player_stats[name]['per_minute'][0][12],
        'FTAp36':player_stats[name]['per_minute'][0][13],
        'ORBp36':player_stats[name]['per_minute'][0][16],
        'DRBp36':player_stats[name]['per_minute'][0][17],
        'TRBp36':player_stats[name]['per_minute'][0][18],
        'ASTp36':player_stats[name]['per_minute'][0][19],
        'TOVp36':player_stats[name]['per_minute'][0][22],
        'STLp36':player_stats[name]['per_minute'][0][20],
        'Blkp36':player_stats[name]['per_minute'][0][21],
        'PFp36':player_stats[name]['per_minute'][0][23],
        'PTSp36':player_stats[name]['per_minute'][0][24],
        'FGp100':player_stats[name]['per_poss'][0][4],
        'FGAp100':player_stats[name]['per_poss'][0][5],
        '3Pp100':player_stats[name]['per_poss'][0][6],
        '3PAp100':player_stats[name]['per_poss'][0][7],
        'FTp100':player_stats[name]['per_poss'][0][12],
        'FTAp100':player_stats[name]['per_poss'][0][13],
        'ORBp100':player_stats[name]['per_poss'][0][16],
        'DRBp100':player_stats[name]['per_poss'][0][17],
        'TRBp100':player_stats[name]['per_poss'][0][18],
        'ASTp100':player_stats[name]['per_poss'][0][19],
        'TOVp100':player_stats[name]['per_poss'][0][22],
        'STLp100':player_stats[name]['per_poss'][0][20],
        'Blkp100':player_stats[name]['per_poss'][0][21],
        'PFp100':player_stats[name]['per_poss'][0][23],
        'PTSp100':player_stats[name]['per_poss'][0][24],
        'PER':player_stats[name]['advanced'][0][3],
        'DPM':player_stats[name]['advanced'][0][17],
        'OPM':player_stats[name]['advanced'][0][16],
        'OWSp36':36*player_stats[name]['advanced'][0][16]/player_stats[name]['advanced'][0][2],
        'DWSp36':36*player_stats[name]['advanced'][0][17]/player_stats[name]['advanced'][0][2]}
    d['3PtP'] = (2/(1+np.exp(-d['3PAp100']))-1)*d['3P%']/100
    d['Creation'] = d['ASTp100']*0.1843+(d['PTSp100']+d['TOVp100'])*0.0969-2.3021*d['3PtP']+0.0582*(d['ASTp100']*(d['PTSp100']+d['TOVp100'])*d['3PtP'] )-1.1942
    d['Load'] = (d['ASTp100']-(0.38*d['Creation'])*0.75)+d['FGAp100']+d['FTAp100']*0.44+d['Creation']+d['TOVp100']
    d['cTOV'] = d['TOVp100']/d['Load']
    #d['DPM'] = d['Blkp100']*0.802+d['DRBp100']*0.42-4.7-0.07551*d['PFp100']+1.597019*d['STLp100']-0.26385*d['TOVp100']
    #d['OPM'] = -8.57647+0.6111*d['PTSp100']-0.33918*(0.44*d['FTAp100']+d['FGAp100'])+0.440814*d['FTAp100']+0.379745*d['3PAp100']+0.634044*d['ASTp100']+0.77827*d['ORBp100']-1.08855*d['TOVp100']+0.26262*d['STLp100']

    #d['PassP'] = ((d['ASTp100']-(0.38*d['Creation']))*0.752+ d['Creation'] + d['TOVp100']) ** 0.67

    player_vectors.append([d[stat] for stat in X.columns])
    if name == 'Ben Simmons':
        print(player_vectors[-1])
X.columns

In [ ]:
d

In [ ]:
#table_columns['per_minute']#,table_columns['advanced'].index('0-3')
#for i,t in enumerate(player_stats[name]['advanced'][0]):
#    print(i,t)


In [ ]:
Xn = np.nan_to_num(np.array(player_vectors))
scalerX2 = preprocessing.StandardScaler()
Xn_s =scalerX2.fit_transform(np.nan_to_num(Xn))
Xn_fs = fexp.transform(np.nan_to_num(Xn_s))
predict = clf.predict(Xn_fs)
ratings = np.nan_to_num(scalery.inverse_transform(predict))
#ratings = scalery.inverse_transform(clf.predict(fexp.transform(scalerX.transform(Xn))))

In [ ]:
#player_vectors[player_names.index('Joel Embiid')][13],X.columns[13]

In [ ]:
X.mean(0)-Xn.mean(0)

In [ ]:
for n,v in zip(X.columns,Xn.min(0)):
    print(n,v)

In [ ]:
rosters = {}

for team in tables:
    for name in tables[team]['roster'].index:
        rosters[name] = team
    #print(team,)

In [ ]:
def load_roster(filename):
    base = json.load(open(filename,'rb'))

    
    for i in range(len(base['players'])):
        player = base['players'][i]
        if 'name' in player and player['name'] !='':
            sname = base['players'][i]['name'].split()
            base['players'][i]['firstName'] = sname[0]
            base['players'][i]['lastName'] = ' '.join(sname[1:])
            del base['players'][i]['name']
    players = base['players']
    base['version'] = 32

    return players,base
players,base = load_roster('normed.json')
abbrev_to_tid = {_['abbrev']:_['tid']for _ in base['teams']}
abbrev_to_tid['PHO'] = abbrev_to_tid['PHX']
abbrev_to_tid['CHO'] = abbrev_to_tid['CHA']
abbrev_to_tid['BRK'] = abbrev_to_tid['BKN']
abbrev_to_tid['NOH'] = abbrev_to_tid['NOP']
abbrev_to_tid['NJN'] = abbrev_to_tid['BKN']
abbrev_to_tid['SEA'] = abbrev_to_tid['OKC']

base['startingSeason'] = tyear

players_discard = [_ for _ in players if  _['tid'] > -3 ]
players = [_ for _ in players if _['tid'] in set([-1,-2,-4,-5])]
y_keys = [_.lower() for _ in y.columns]

y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }

In [ ]:
import pprint
import copy

pset2 = copy.deepcopy(players)
pset3 = copy.deepcopy(players)
pp = pprint.PrettyPrinter()
for i,name in enumerate(player_names):
    name = player_names[i]
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])

    new_player['tid'] = abbrev_to_tid[rosters[name]] if name in rosters else -1
    if name in rosters:
        new_player['pos'] = player_stats[name]['roster'][0][0]
        new_player['weight'] = int(player_stats[name]['roster'][0][2])
        ht = [int(_) for _ in player_stats[name]['roster'][0][1].split('-')]
        new_player['height'] = ht[0]*12 + ht[1]
        new_player['born'] = {'year':int(player_stats[name]['roster'][0][3].split(',')[-1]),'loc':''}
        if tyear>=2019 and 'contracts' in player_stats[name]:
            ctrct = [float(_) for _ in player_stats[name]['contracts'][0] if _ != '0.5']
            if len(ctrct) > 0:
                new_player['contract'] = {'amount':int(round(sum(ctrct)/(1000*len(ctrct)))),'exp':2018+len(ctrct)}
    else:
        continue

    new_player['ratings'] = [{k: int(np.clip(ratings[i,y_keys.index(km)],0,100)) for k,km in y_map.items()}]
    
    players.append(new_player)
    if name in ['LeBron James', "Trevon Duval"]:
        pp.pprint(new_player)
        #pp.pprint(player_vectors[player_names.index(name)])

In [ ]:
if False:
    ri = 0
    _ = plt.hist(yt[:,ri],100,normed=True,alpha=0.5,label='predicted')
    _ = plt.hist(y.iloc[:,ri],100,normed=True,alpha=0.5,label='labels')
    _ = plt.hist(ratings[:,ri],100,normed=True,alpha=0.5,label='dataset')
    plt.xlim(-100,100)
    plt.legend()
elif False:
    plt.figure(figsize=(20,20))

    for ri in range(len(X.columns)):
        #ri = 13
        plt.figure()
        #plt.subplot(6,6,ri+1)

        _ = plt.hist(Xn[:,ri],100,density=True,alpha=0.5,label='mine')
        _ = plt.hist(X.iloc[:,ri],100,density=True,alpha=0.5,label='expected')
        plt.legend()
        plt.title(X.columns[ri] + ' ' + str(ri))
        plt.tight_layout()
        plt.xlim(-100,100)


In [ ]:
base['players'] = players
with open('auto_roster_{}.json'.format(tyear),'wt') as fp:
    json.dump(base,fp, indent=4, sort_keys=True)

In [ ]:
import scipy

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

bpms = []
for name in player_stats:
    if 'advanced' in player_stats[name]:
        bpm = 0#player_stats[name]['advanced'][0][23]
        vorp = player_stats[name]['advanced'][0][24]
        bpms.append((bpm+vorp,name))
i=0
m = len(bpms)
for v,k in sorted(bpms):
    player_stats[k]['bpm'] = ((i)/m)*0.8+0.2
    i+=1
bpms = np.array([_[0] for _ in bpms])
bm,bs = bpms.mean(),bpms.std()
bm,bs

In [ ]:
player_stats["LeBron James"]['bpm']

In [ ]:
lkc = [_.lower() for _ in df._get_numeric_data().columns]
dfm = {k:v for k,v in zip(lkc,df.mean(0))}
dfs = {k:v for k,v in zip(lkc,df.std(0))}

In [ ]:
player_vectors2 = []
player_names2 = []
ok = sorted(y_map.keys())
bpmv = []
for name in player_stats:
    if not 'totals' in player_stats[name]:
        continue
    if 'roster' in  player_stats[name]:
        ht = [int(_) for _ in player_stats[name]['roster'][0][1].split('-')]
        h2 = ht[0]*12 + ht[1]
        w = int(player_stats[name]['roster'][0][2])
    else:
        continue
    player_names2.append(name)
    BPM = player_stats[name]['bpm']
    bpmv.append(BPM)
    d=    {
        'hgt':h2,
        'stre':w,
        'spd':player_stats[name]['advanced'][0][13],
        'jmp':player_stats[name]['per_minute'][0][23],
        'endu':player_stats[name]['advanced'][0][14],
        'ins':player_stats[name]['per_minute'][0][13],
        'dnk':player_stats[name]['shooting'][0][11]*100,
        'ft':player_stats[name]['totals'][0][16], #ft% times made
        'fg':player_stats[name]['advanced'][0][4], #ts
        'tp':player_stats[name]['totals'][0][9], #3% times made
        'diq':player_stats[name]['advanced'][0][22],
        'oiq':player_stats[name]['advanced'][0][21],
        'drb':player_stats[name]['advanced'][0][11]*player_stats[name]['per_minute'][0][22],
        'pss':player_stats[name]['advanced'][0][10]*player_stats[name]['per_minute'][0][19],
        'reb':player_stats[name]['advanced'][0][9]*player_stats[name]['per_minute'][0][18]}
    player_vectors2.append([d[k]*BPM for k in sorted(d)])
    if name == 'LeBron James':
        print(player_vectors2[-1])
#print(player_vectors2[player_names2.index('Ben Simmons')])

#player_vectors3 = [k:[v[_] for _ in sorted(v)] for k,v in player_vectors2.items() }
Xman = np.array(player_vectors2)#*np.array(bpmv)[:,np.newaxis]#['Ben Simmons']
Xman = np.nan_to_num(Xman)
Xmn = (Xman -Xman.mean(0))/Xman.std(0)

player_man = {}

for n,r in zip(player_names2,Xmn):
    #print(n,r)
    try:
        player_man[n] = {k:int(np.clip(r[i]*dfs[y_map[k]]+dfm[y_map[k]],0,100)) for i,k in enumerate(ok)}
    except:
        print(n)
        print(r)
        raise

In [ ]:
name = "LeBron James"
player_stats[name]['bpm']
_ = plt.hist(Xmn[:,3],8)
Xmn[player_names2.index('LeBron James'),9]

In [ ]:
Xman.mean(0)

In [ ]:
player_man[name]

In [ ]:
for i,name in enumerate(player_names):
    name = player_names[i]
    if name not in player_man:
        continue
    sname = name.split(' ')
    
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])

    new_player['tid'] = abbrev_to_tid[rosters[name]] if name in rosters else -1
    if name in rosters:
        new_player['pos'] = player_stats[name]['roster'][0][0]
        new_player['weight'] = int(player_stats[name]['roster'][0][2])
        ht = [int(_) for _ in player_stats[name]['roster'][0][1].split('-')]
        new_player['height'] = ht[0]*12 + ht[1]
        new_player['born'] = {'year':int(player_stats[name]['roster'][0][3].split(',')[-1]),'loc':''}
        if tyear >= 2019 and 'contracts' in player_stats[name]:
            ctrct = [float(_) for _ in player_stats[name]['contracts'][0] if _ != '0.5']
            if len(ctrct) > 0:
                new_player['contract'] = {'amount':int(round(sum(ctrct)/(1000*len(ctrct)))),'exp':2018+len(ctrct)}
    else:
        continue

    new_player['ratings'] = [player_man[name]]
    
    pset2.append(new_player)
    if name in ['LeBron James', "Trevon Duval"]:
        pp.pprint(new_player)

In [ ]:
base['players'] = pset2
with open('auto_roster_nomodel.json','wt') as fp:
    json.dump(base,fp, indent=4, sort_keys=True)

In [ ]:
 2*player_stats['LeBron James']['bpm']


In [ ]:
pset4 = copy.deepcopy(pset3)
for i,name in enumerate(player_names):
    name = player_names[i]
    sname = name.split(' ')
    new_player = {}
    new_player['firstName'] = sname[0]
    new_player['lastName'] = ' '.join(sname[1:])

    new_player['tid'] = abbrev_to_tid[rosters[name]] if name in rosters else -1
    if name in rosters:

        new_player['pos'] = player_stats[name]['roster'][0][0]
        new_player['weight'] = int(player_stats[name]['roster'][0][2])
        ht = [int(_) for _ in player_stats[name]['roster'][0][1].split('-')]
        new_player['height'] = ht[0]*12 + ht[1]
        new_player['born'] = {'year':int(player_stats[name]['roster'][0][3].split(',')[-1]),'loc':''}
        if tyear >= 2019 and 'contracts' in player_stats[name]:
            ctrct = [float(_) for _ in player_stats[name]['contracts'][0] if _ != '0.5']
            if len(ctrct) > 0:
                new_player['contract'] = {'amount':int(round(sum(ctrct)/(1000*len(ctrct)))),'exp':2018+len(ctrct)}
    else:
        continue
    BPM = sigmoid((player_stats[name]['advanced'][0][24]-bm)/(40*bs))*2

    new_player['ratings'] = [{k: int(np.clip(BPM*ratings[i,y_keys.index(km)],0,100)) for k,km in y_map.items()}]
    
    pset4.append(new_player)

In [ ]:
base['players'] = pset4
with open('auto_roster_model_vorp.json','wt') as fp:
    json.dump(base,fp, indent=4, sort_keys=True)

In [ ]:
sigmoid((player_stats['LeBron James']['advanced'][0][24]-bm)/(30*bs))*2,sigmoid((player_stats['James Harden']['advanced'][0][24]-bm)/(30*bs))*2

